In [138]:
import pandas as pd
import psycopg2
import sys
from psycopg2 import OperationalError, errorcodes, errors
import psycopg2.extras as extras

In [189]:
# Read data into a Pandas DataFrames

train = pd.read_csv("Resources/2023-01-29-updated_train.csv")
lotshape_table = pd.read_csv("Resources/LotShape.csv")
garagetype_table = pd.read_csv("Resources/GarageType.csv")
heating_table = pd.read_csv("Resources/Heating.csv")
housetype_table = pd.read_csv("Resources/HouseType.csv")
neighborhood_table = pd.read_csv("Resources/Neighborhood.csv")

In [140]:
# Display the data

train.head()

,LotArea,LotShape,BldgType,YearBuilt,Neighborhood,YearRemodAdd,Heating,TtlHouseSF,FullBath,HalfBath,BedroomAbvGr,Fireplaces,GarageType,GarageCars,PavedDrive,PoolArea,DateSold,SalePrice
0,8450,Reg,1Fam,2003,CollgCr,2003,GasA,2566,2,1,3,0,Attchd,2,Y,0,2008-2-1,208500
1,9600,Reg,1Fam,1976,Veenker,1976,GasA,2524,2,0,3,1,Attchd,2,Y,0,2007-5-1,181500
2,11250,IR1,1Fam,2001,CollgCr,2002,GasA,2706,2,1,3,1,Attchd,2,Y,0,2008-9-1,223500
3,9550,IR1,1Fam,1915,Crawfor,1970,GasA,2473,1,0,3,1,Detchd,3,Y,0,2006-2-1,140000
4,14260,IR1,1Fam,2000,NoRidge,2000,GasA,3343,2,1,4,1,Attchd,3,Y,0,2008-12-1,250000


In [141]:
# Check for null values

train.isnull().sum()

LotArea          0
LotShape         0
BldgType         0
YearBuilt        0
Neighborhood     0
YearRemodAdd     0
Heating          0
TtlHouseSF       0
FullBath         0
HalfBath         0
BedroomAbvGr     0
Fireplaces       0
GarageType      78
GarageCars       0
PavedDrive       0
PoolArea         0
DateSold         0
SalePrice        0
dtype: int64

In [142]:
# Read the data from the neighborhood dataset

neighborhood_df = pd.read_csv("Resources/Neighborhood_Dataset.csv")

In [143]:
# Display the neighborhood dataset

neighborhood_df.head()

,Neighborhood,Column2
0,Blmngtn,Bloomington Heights
1,Blueste,Bluestem
2,BrDale,Briardale
3,BrkSide,Brookside
4,ClearCr,Clear Creek


In [144]:
# Merge neighborhood dataframe with train datadrame

updated_train = pd.merge(train, neighborhood_df, on = "Neighborhood", how = "inner")

updated_train.head()

,LotArea,LotShape,BldgType,YearBuilt,Neighborhood,YearRemodAdd,Heating,TtlHouseSF,FullBath,HalfBath,BedroomAbvGr,Fireplaces,GarageType,GarageCars,PavedDrive,PoolArea,DateSold,SalePrice,Column2
0,8450,Reg,1Fam,2003,CollgCr,2003,GasA,2566,2,1,3,0,Attchd,2,Y,0,2008-2-1,208500,College Creek
1,11250,IR1,1Fam,2001,CollgCr,2002,GasA,2706,2,1,3,1,Attchd,2,Y,0,2008-9-1,223500,College Creek
2,10652,IR1,1Fam,2006,CollgCr,2007,GasA,2988,2,0,3,1,Attchd,3,Y,0,2007-8-1,279500,College Creek
3,9742,Reg,1Fam,2002,CollgCr,2002,GasA,3572,2,0,3,1,Attchd,2,Y,0,2008-9-1,230000,College Creek
4,11049,Reg,1Fam,2007,CollgCr,2007,GasA,2468,2,0,3,0,Attchd,2,Y,0,2008-1-1,179900,College Creek


In [145]:
# Drop original neighborhood column

final_train = updated_train.drop(["Neighborhood"], axis=1)

final_train.head()

,LotArea,LotShape,BldgType,YearBuilt,YearRemodAdd,Heating,TtlHouseSF,FullBath,HalfBath,BedroomAbvGr,Fireplaces,GarageType,GarageCars,PavedDrive,PoolArea,DateSold,SalePrice,Column2
0,8450,Reg,1Fam,2003,2003,GasA,2566,2,1,3,0,Attchd,2,Y,0,2008-2-1,208500,College Creek
1,11250,IR1,1Fam,2001,2002,GasA,2706,2,1,3,1,Attchd,2,Y,0,2008-9-1,223500,College Creek
2,10652,IR1,1Fam,2006,2007,GasA,2988,2,0,3,1,Attchd,3,Y,0,2007-8-1,279500,College Creek
3,9742,Reg,1Fam,2002,2002,GasA,3572,2,0,3,1,Attchd,2,Y,0,2008-9-1,230000,College Creek
4,11049,Reg,1Fam,2007,2007,GasA,2468,2,0,3,0,Attchd,2,Y,0,2008-1-1,179900,College Creek


In [146]:
# Rename 'column2' to 'Neighborhood'

final_train.rename(columns = {'Column2':'Neighborhood'}, inplace = True)

In [147]:
# Display work in progress

final_train

,LotArea,LotShape,BldgType,YearBuilt,YearRemodAdd,Heating,TtlHouseSF,FullBath,HalfBath,BedroomAbvGr,Fireplaces,GarageType,GarageCars,PavedDrive,PoolArea,DateSold,SalePrice,Neighborhood
0,8450,Reg,1Fam,2003,2003,GasA,2566,2,1,3,0,Attchd,2,Y,0,2008-2-1,208500,College Creek
1,11250,IR1,1Fam,2001,2002,GasA,2706,2,1,3,1,Attchd,2,Y,0,2008-9-1,223500,College Creek
2,10652,IR1,1Fam,2006,2007,GasA,2988,2,0,3,1,Attchd,3,Y,0,2007-8-1,279500,College Creek
3,9742,Reg,1Fam,2002,2002,GasA,3572,2,0,3,1,Attchd,2,Y,0,2008-9-1,230000,College Creek
4,11049,Reg,1Fam,2007,2007,GasA,2468,2,0,3,0,Attchd,2,Y,0,2008-1-1,179900,College Creek
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1146,9842,Reg,1Fam,1921,1998,GasA,3213,3,1,4,0,BuiltIn,2,Y,0,2008-5-1,189000,South & West of Iowa State University
1147,6292,Reg,1Fam,1930,1950,GasA,1558,1,0,2,0,Detchd,1,Y,0,2008-4-1,91000,South & West of Iowa State University
1148,6171,Reg,1Fam,1925,1990,GasA,2584,2,1,3,1,Detchd,1,Y,0,2009-10-1,137450,South & West of Iowa State University
1149,1950,Reg,Twnhs,1980,1980,GasA,2272,2,1,3,1,Attchd,2,Y,0,2008-7-1,151000,Bluestem


In [148]:
# Move new neighborhood column to original position

colNeighbor = final_train.pop('Neighborhood')

final_train.insert(4, 'Neighborhood', colNeighbor)

final_train

,LotArea,LotShape,BldgType,YearBuilt,Neighborhood,YearRemodAdd,Heating,TtlHouseSF,FullBath,HalfBath,BedroomAbvGr,Fireplaces,GarageType,GarageCars,PavedDrive,PoolArea,DateSold,SalePrice
0,8450,Reg,1Fam,2003,College Creek,2003,GasA,2566,2,1,3,0,Attchd,2,Y,0,2008-2-1,208500
1,11250,IR1,1Fam,2001,College Creek,2002,GasA,2706,2,1,3,1,Attchd,2,Y,0,2008-9-1,223500
2,10652,IR1,1Fam,2006,College Creek,2007,GasA,2988,2,0,3,1,Attchd,3,Y,0,2007-8-1,279500
3,9742,Reg,1Fam,2002,College Creek,2002,GasA,3572,2,0,3,1,Attchd,2,Y,0,2008-9-1,230000
4,11049,Reg,1Fam,2007,College Creek,2007,GasA,2468,2,0,3,0,Attchd,2,Y,0,2008-1-1,179900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1146,9842,Reg,1Fam,1921,South & West of Iowa State University,1998,GasA,3213,3,1,4,0,BuiltIn,2,Y,0,2008-5-1,189000
1147,6292,Reg,1Fam,1930,South & West of Iowa State University,1950,GasA,1558,1,0,2,0,Detchd,1,Y,0,2008-4-1,91000
1148,6171,Reg,1Fam,1925,South & West of Iowa State University,1990,GasA,2584,2,1,3,1,Detchd,1,Y,0,2009-10-1,137450
1149,1950,Reg,Twnhs,1980,Bluestem,1980,GasA,2272,2,1,3,1,Attchd,2,Y,0,2008-7-1,151000


In [149]:
# Making a copy of dataset for cleaning purposes

cleaning_data = final_train.copy()

cleaning_data

,LotArea,LotShape,BldgType,YearBuilt,Neighborhood,YearRemodAdd,Heating,TtlHouseSF,FullBath,HalfBath,BedroomAbvGr,Fireplaces,GarageType,GarageCars,PavedDrive,PoolArea,DateSold,SalePrice
0,8450,Reg,1Fam,2003,College Creek,2003,GasA,2566,2,1,3,0,Attchd,2,Y,0,2008-2-1,208500
1,11250,IR1,1Fam,2001,College Creek,2002,GasA,2706,2,1,3,1,Attchd,2,Y,0,2008-9-1,223500
2,10652,IR1,1Fam,2006,College Creek,2007,GasA,2988,2,0,3,1,Attchd,3,Y,0,2007-8-1,279500
3,9742,Reg,1Fam,2002,College Creek,2002,GasA,3572,2,0,3,1,Attchd,2,Y,0,2008-9-1,230000
4,11049,Reg,1Fam,2007,College Creek,2007,GasA,2468,2,0,3,0,Attchd,2,Y,0,2008-1-1,179900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1146,9842,Reg,1Fam,1921,South & West of Iowa State University,1998,GasA,3213,3,1,4,0,BuiltIn,2,Y,0,2008-5-1,189000
1147,6292,Reg,1Fam,1930,South & West of Iowa State University,1950,GasA,1558,1,0,2,0,Detchd,1,Y,0,2008-4-1,91000
1148,6171,Reg,1Fam,1925,South & West of Iowa State University,1990,GasA,2584,2,1,3,1,Detchd,1,Y,0,2009-10-1,137450
1149,1950,Reg,Twnhs,1980,Bluestem,1980,GasA,2272,2,1,3,1,Attchd,2,Y,0,2008-7-1,151000


In [150]:
# Converting 'PoolArea' column from integer to boolean

cleaning_data['PoolArea'] = cleaning_data['PoolArea'].astype('bool')

# Convert 'Fireplaces' column from integer to Boolean

cleaning_data['Fireplaces'] = cleaning_data['Fireplaces'].astype('bool')

In [151]:
# Making sure data type was changed successfully

cleaning_data.dtypes

LotArea          int64
LotShape        object
BldgType        object
YearBuilt        int64
Neighborhood    object
YearRemodAdd     int64
Heating         object
TtlHouseSF       int64
FullBath         int64
HalfBath         int64
BedroomAbvGr     int64
Fireplaces        bool
GarageType      object
GarageCars       int64
PavedDrive      object
PoolArea          bool
DateSold        object
SalePrice        int64
dtype: object

In [152]:
# Visualize Data

cleaning_data.head(30)

,LotArea,LotShape,BldgType,YearBuilt,Neighborhood,YearRemodAdd,Heating,TtlHouseSF,FullBath,HalfBath,BedroomAbvGr,Fireplaces,GarageType,GarageCars,PavedDrive,PoolArea,DateSold,SalePrice
0,8450,Reg,1Fam,2003,College Creek,2003,GasA,2566,2,1,3,False,Attchd,2,Y,False,2008-2-1,208500
1,11250,IR1,1Fam,2001,College Creek,2002,GasA,2706,2,1,3,True,Attchd,2,Y,False,2008-9-1,223500
2,10652,IR1,1Fam,2006,College Creek,2007,GasA,2988,2,0,3,True,Attchd,3,Y,False,2007-8-1,279500
3,9742,Reg,1Fam,2002,College Creek,2002,GasA,3572,2,0,3,True,Attchd,2,Y,False,2008-9-1,230000
4,11049,Reg,1Fam,2007,College Creek,2007,GasA,2468,2,0,3,False,Attchd,2,Y,False,2008-1-1,179900
5,10859,Reg,1Fam,1994,College Creek,1995,GasA,2194,1,1,3,False,Attchd,2,Y,False,2009-6-1,145000
6,9200,IR1,1Fam,1975,College Creek,1980,GasA,1876,1,0,3,False,Detchd,1,Y,False,2008-7-1,130250
7,11645,IR1,1Fam,2004,College Creek,2004,GasA,2580,2,1,3,False,Attchd,2,Y,False,2006-8-1,196500
8,7200,Reg,1Fam,1972,College Creek,1972,GasA,1560,1,0,2,False,Detchd,1,Y,False,2008-1-1,124900
9,9375,Reg,1Fam,1997,College Creek,1998,GasA,3091,2,1,3,False,Attchd,2,Y,False,2009-2-1,219500


In [153]:
# Check for null values

cleaning_data.isnull().sum()

LotArea          0
LotShape         0
BldgType         0
YearBuilt        0
Neighborhood     0
YearRemodAdd     0
Heating          0
TtlHouseSF       0
FullBath         0
HalfBath         0
BedroomAbvGr     0
Fireplaces       0
GarageType      72
GarageCars       0
PavedDrive       0
PoolArea         0
DateSold         0
SalePrice        0
dtype: int64

In [154]:
# Replace NaN values with 'NoGar' for No Garage

cleaning_data['GarageType'] = cleaning_data['GarageType'].fillna('NoGar').astype(str)

In [155]:
# Check for null values

cleaning_data.isnull().sum()

LotArea         0
LotShape        0
BldgType        0
YearBuilt       0
Neighborhood    0
YearRemodAdd    0
Heating         0
TtlHouseSF      0
FullBath        0
HalfBath        0
BedroomAbvGr    0
Fireplaces      0
GarageType      0
GarageCars      0
PavedDrive      0
PoolArea        0
DateSold        0
SalePrice       0
dtype: int64

In [156]:
# Visualize data

cleaning_data.head(20)

,LotArea,LotShape,BldgType,YearBuilt,Neighborhood,YearRemodAdd,Heating,TtlHouseSF,FullBath,HalfBath,BedroomAbvGr,Fireplaces,GarageType,GarageCars,PavedDrive,PoolArea,DateSold,SalePrice
0,8450,Reg,1Fam,2003,College Creek,2003,GasA,2566,2,1,3,False,Attchd,2,Y,False,2008-2-1,208500
1,11250,IR1,1Fam,2001,College Creek,2002,GasA,2706,2,1,3,True,Attchd,2,Y,False,2008-9-1,223500
2,10652,IR1,1Fam,2006,College Creek,2007,GasA,2988,2,0,3,True,Attchd,3,Y,False,2007-8-1,279500
3,9742,Reg,1Fam,2002,College Creek,2002,GasA,3572,2,0,3,True,Attchd,2,Y,False,2008-9-1,230000
4,11049,Reg,1Fam,2007,College Creek,2007,GasA,2468,2,0,3,False,Attchd,2,Y,False,2008-1-1,179900
5,10859,Reg,1Fam,1994,College Creek,1995,GasA,2194,1,1,3,False,Attchd,2,Y,False,2009-6-1,145000
6,9200,IR1,1Fam,1975,College Creek,1980,GasA,1876,1,0,3,False,Detchd,1,Y,False,2008-7-1,130250
7,11645,IR1,1Fam,2004,College Creek,2004,GasA,2580,2,1,3,False,Attchd,2,Y,False,2006-8-1,196500
8,7200,Reg,1Fam,1972,College Creek,1972,GasA,1560,1,0,2,False,Detchd,1,Y,False,2008-1-1,124900
9,9375,Reg,1Fam,1997,College Creek,1998,GasA,3091,2,1,3,False,Attchd,2,Y,False,2009-2-1,219500


In [157]:
# Export as CSV

cleaning_data.to_csv('Clean_Data.csv', header=True, index=False)

In [158]:
# cleaning_data.corr()

Creating the database, tables, and importing data

In [159]:
conn_params_dic =   {"host": "localhost",
                    "user": "postgres",
                    "password": "8kHFtf@nb",
                    # port="5432"
                    }

In [160]:
def show_psycopg2_exception(err):
    # Get details about exception
    err_type, err_obj, traceback = sys.exc_info()

    # Get the line number when exception occured
    line_n = traceback.tb_lineno

    # Print the connect() error
    print ("\npsycopg2 ERROR:", err, "on line number:", line_n)
    print ("psycopg2 traceback:", traceback, "-- type:", err_type)

    # psycopg2 extensions.Diagnostics object attribute
    print ("\nextensions.Diagnostics", err.diag)

    # Print the pgcode and pgerror exceptions
    print ("pgerror:", err.pgerror)
    print ("pgcode:", err.pgcode, "\n")

In [161]:
# Define a connect function to connect to PostreSQL database server
def connect(conn_params_dic):
    conn = None
    try:
        print('Connecting to the PostreSQL.....')
        conn = psycopg2.connect(**conn_params_dic)
        print("Connection Successful.....")

    except OperationalError as err:
        # Passing exception to function
        show_psycopg2_exception(err)

        # Set the connection to 'None' in case of error
        conn = None
    
    return conn

In [162]:
# Creates a database named HouseSaleHistory in PostreSQL database

conn = connect(conn_params_dic)
conn.autocommit = True

if conn!=None:

    try:
        cursor = conn.cursor();
        # Dropping database HouseSaleHistory if exists
        cursor.execute("DROP DATABASE IF EXISTS HouseSaleHistory;")

        # Creating a database
        cursor.execute("CREATE DATABASE HouseSaleHistory;")
        print("HouseSaleHistory database is created successfully.....")

        # Closing the cursor & connection
        cursor.close()
        conn.close()

    except OperationalError as err:
        # Pass exception to function
        show_psycopg2_exception(err)
        # Set the connection to 'None' in case of error
        conn = None

Connecting to the PostreSQL.....
Connection Successful.....
HouseSaleHistory database is created successfully.....


In [163]:
# Create a table named housesalehistory in PostgreSQL database

conn = connect(conn_params_dic)
conn.autocommit = True

if conn!=None:

    try:
        cursor = conn.cursor()
        # Dropping table HouseSaleHistory if exists
        cursor.execute("DROP TABLE IF EXISTS HouseSaleHistory;")

        sql = '''
        CREATE TABLE HouseSaleHistory(
        LotArea numeric,
        LotShape varchar(10),
        BldgType varchar(10),
        YearBuilt numeric,
        Neighborhood varchar(50),
        YearRemodAdd numeric,
        Heating varchar(10),
        TtlHouseSF numeric,
        FullBath int,
        HalfBath int,
        BedroomAbvGr int,
        Fireplaces boolean,
        GarageType varchar(10),
        GarageCars int,
        PavedDrive varchar(10),
        PoolArea boolean,
        DateSold date,
        SalePrice numeric
        )'''

        # Creating a table
        cursor.execute(sql)
        print("HouseSaleHistory table has been created successfully.....")

        # Closing the cursor & connection
        cursor.close()
        conn.close()

    except OperationalError as err:
        # Pass exception to function
        show_psycopg2_exception(err)
        # Set the connection to 'None' in case of error
        conn = None
    

Connecting to the PostreSQL.....
Connection Successful.....
HouseSaleHistory table has been created successfully.....


In [164]:
# Define function using cursor.executemany() to insert the dataframe

def execute_many(conn, datafrm, table):

    # Creating a list of tupples from the dataframe values
    tpls = [tuple(x) for x in datafrm.to_numpy()]

    # Dataframe columns with comma-separated
    cols = ','.join(list(datafrm.columns))

    # SQL Query to execute
    sql = "INSERT INTO %s(%s) VALUES (%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s)" % (table, cols)
    cursor = conn.cursor()
    try:
        cursor.executemany(sql, tpls)
        conn.commit()
        print("Data inserted using execute_many() successfully...")
    except (Exception, psycopg2.DatabaseError) as err:
        # Pass exception to function
        show_psycopg2_exception(err)
        cursor.close()

# Connect to the database
conn = connect(conn_params_dic)
conn.autocommit = True

# Run the execute_many method
execute_many(conn, cleaning_data, 'housesalehistory')

Connecting to the PostreSQL.....
Connection Successful.....
Data inserted using execute_many() successfully...


In [172]:
# Create a table named lotshape in PostgreSQL database

conn = connect(conn_params_dic)
conn.autocommit = True

if conn!=None:

    try:
        cursor = conn.cursor()
        # Dropping table lotshape if exists
        cursor.execute("DROP TABLE IF EXISTS lotshape;")

        lotshape = '''
        CREATE TABLE lotshape (
        id varchar(10) PRIMARY KEY,
        lotshape_desc varchar(100)
        )'''

        # Creating a table
        cursor.execute(lotshape)
        print("Lotshape table has been created successfully.....")

        # Closing the cursor & connection
        cursor.close()
        conn.close()

    except OperationalError as err:
        # Pass exception to function
        show_psycopg2_exception(err)
        # Set the connection to 'None' in case of error
        conn = None
    

Connecting to the PostreSQL.....
Connection Successful.....
Lotshape table has been created successfully.....


In [166]:
# Define function using cursor.executemany() to insert the dataframe

def execute_many(conn, datafrm, table):

    # Creating a list of tupples from the dataframe values
    tpls = [tuple(x) for x in datafrm.to_numpy()]

    # Dataframe columns with comma-separated
    cols = ','.join(list(datafrm.columns))

    # SQL Query to execute
    LotShape = "INSERT INTO %s(%s) VALUES (%%s,%%s)" % (table, cols)
    cursor = conn.cursor()
    try:
        cursor.executemany(LotShape, tpls)
        conn.commit()
        print("Data inserted using execute_many() successfully...")
    except (Exception, psycopg2.DatabaseError) as err:
        # Pass exception to function
        show_psycopg2_exception(err)
        cursor.close()

# Connect to the database
conn = connect(conn_params_dic)
conn.autocommit = True

# Run the execute_many method
execute_many(conn, lotshape_table, 'lotshape')

Connecting to the PostreSQL.....
Connection Successful.....
Data inserted using execute_many() successfully...


In [167]:
# Create a table named neighborhood in PostgreSQL database

conn = connect(conn_params_dic)
conn.autocommit = True

if conn!=None:

    try:
        cursor = conn.cursor()
        # Dropping table neighborhood if exists
        cursor.execute("DROP TABLE IF EXISTS neighborhood;")

        neighborhood = '''
        CREATE TABLE "neighborhood" (
        "id" varchar(10) PRIMARY KEY,
        "neighborhood_name" varchar(100)
        )'''

        # Creating a table
        cursor.execute(neighborhood)
        print("neighborhood table has been created successfully.....")

        # Closing the cursor & connection
        cursor.close()
        conn.close()

    except OperationalError as err:
        # Pass exception to function
        show_psycopg2_exception(err)
        # Set the connection to 'None' in case of error
        conn = None

Connecting to the PostreSQL.....
Connection Successful.....
neighborhood table has been created successfully.....


In [168]:
# Define function using cursor.executemany() to insert the dataframe

def execute_many(conn, datafrm, table):

    # Creating a list of tupples from the dataframe values
    tpls = [tuple(x) for x in datafrm.to_numpy()]

    # Dataframe columns with comma-separated
    cols = ','.join(list(datafrm.columns))

    # SQL Query to execute
    Neighborhood = "INSERT INTO %s(%s) VALUES (%%s,%%s)" % (table, cols)
    cursor = conn.cursor()
    try:
        cursor.executemany(Neighborhood, tpls)
        conn.commit()
        print("Data inserted using execute_many() successfully...")
    except (Exception, psycopg2.DatabaseError) as err:
        # Pass exception to function
        show_psycopg2_exception(err)
        cursor.close()

# Connect to the database
conn = connect(conn_params_dic)
conn.autocommit = True

# Run the execute_many method
execute_many(conn, neighborhood_table, 'neighborhood')

Connecting to the PostreSQL.....
Connection Successful.....
Data inserted using execute_many() successfully...


In [180]:
# Create a table named heating in PostgreSQL database

conn = connect(conn_params_dic)
conn.autocommit = True

if conn!=None:

    try:
        cursor = conn.cursor()
        # Dropping table heating if exists
        cursor.execute("DROP TABLE IF EXISTS heating;")

        heating = '''
        CREATE TABLE "heating" (
        "id" varchar(10) PRIMARY KEY,
        "heat_desc" varchar(100)
        )'''

        # Creating a table
        cursor.execute(heating)
        print("heating table has been created successfully.....")

        # Closing the cursor & connection
        cursor.close()
        conn.close()

    except OperationalError as err:
        # Pass exception to function
        show_psycopg2_exception(err)
        # Set the connection to 'None' in case of error
        conn = None

Connecting to the PostreSQL.....
Connection Successful.....
heating table has been created successfully.....


In [181]:
# Define function using cursor.executemany() to insert the dataframe

def execute_many(conn, datafrm, table):

    # Creating a list of tupples from the dataframe values
    tpls = [tuple(x) for x in datafrm.to_numpy()]

    # Dataframe columns with comma-separated
    cols = ','.join(list(datafrm.columns))

    # SQL Query to execute
    Heating = "INSERT INTO %s(%s) VALUES (%%s,%%s)" % (table, cols)
    cursor = conn.cursor()
    try:
        cursor.executemany(Heating, tpls)
        conn.commit()
        print("Data inserted using execute_many() successfully...")
    except (Exception, psycopg2.DatabaseError) as err:
        # Pass exception to function
        show_psycopg2_exception(err)
        cursor.close()

# Connect to the database
conn = connect(conn_params_dic)
conn.autocommit = True

# Run the execute_many method
execute_many(conn, heating_table, 'heating')

Connecting to the PostreSQL.....
Connection Successful.....
Data inserted using execute_many() successfully...


In [182]:
# Create a table named garagetype in PostgreSQL database

conn = connect(conn_params_dic)
conn.autocommit = True

if conn!=None:

    try:
        cursor = conn.cursor()
        # Dropping table garagetype if exists
        cursor.execute("DROP TABLE IF EXISTS garagetype;")

        garagetype = '''
        CREATE TABLE "garagetype" (
        "id" varchar(10) PRIMARY KEY,
        "gar_desc" varchar(100)
        )'''

        # Creating a table
        cursor.execute(garagetype)
        print("garagetype table has been created successfully.....")

        # Closing the cursor & connection
        cursor.close()
        conn.close()

    except OperationalError as err:
        # Pass exception to function
        show_psycopg2_exception(err)
        # Set the connection to 'None' in case of error
        conn = None

Connecting to the PostreSQL.....
Connection Successful.....
garagetype table has been created successfully.....


In [190]:
# Define function using cursor.executemany() to insert the dataframe

def execute_many(conn, datafrm, table):

    # Creating a list of tupples from the dataframe values
    tpls = [tuple(x) for x in datafrm.to_numpy()]

    # Dataframe columns with comma-separated
    cols = ','.join(list(datafrm.columns))

    # SQL Query to execute
    GarageType = "INSERT INTO %s(%s) VALUES (%%s,%%s)" % (table, cols)
    cursor = conn.cursor()
    try:
        cursor.executemany(GarageType, tpls)
        conn.commit()
        print("Data inserted using execute_many() successfully...")
    except (Exception, psycopg2.DatabaseError) as err:
        # Pass exception to function
        show_psycopg2_exception(err)
        cursor.close()

# Connect to the database
conn = connect(conn_params_dic)
conn.autocommit = True

# Run the execute_many method
execute_many(conn, garagetype_table, 'garagetype')

Connecting to the PostreSQL.....
Connection Successful.....

psycopg2 ERROR: syntax error at or near "desc"
LINE 1: INSERT INTO garagetype(id,desc) VALUES ('2Types','More than ...
                                  ^
 on line number: 15
psycopg2 traceback: <traceback object at 0x0000022B4124A400> -- type: <class 'psycopg2.errors.SyntaxError'>

extensions.Diagnostics <psycopg2.extensions.Diagnostics object at 0x0000022B414A0650>
pgerror: ERROR:  syntax error at or near "desc"
LINE 1: INSERT INTO garagetype(id,desc) VALUES ('2Types','More than ...
                                  ^

pgcode: 42601 

